<a href="https://colab.research.google.com/github/jajapuramshivasai/iquhack_2025/blob/main/Qring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2025-Quantum-Factorization-With-Quantum-Rings


# Shor's Algorithm
---------


## Step 1: Finding a Suitable Base

Choose a random integer a such that 1 < a < N.
Calculate the greatest common divisor (GCD) of a and N.
If GCD(a, N) ≠ 1, then GCD(a, N) is a non-trivial factor of N, and the algorithm terminates.
If GCD(a, N) = 1, proceed to Step 2.


## Step 2: Period Finding

Determine the period (r) of the function f(x) = aˣ mod N. This is the smallest positive integer r such that aʳ mod N = 1.
If r is odd, return to Step 1 with a different random a.
If r is even and a^(r/2) + 1 ≡ 0 (mod N), return to Step 1 with a different random a.
If r is even and a^(r/2) + 1 <binary data, 1 bytes><binary data, 1 bytes><binary data, 1 bytes> 0 (mod N), proceed to Step 3.

## Step 3: Factor Calculation

Since aʳ ≡ 1 (mod N) and r is even, we can write aʳ - 1 = kN for some integer k. This can be factored as (a^(r/2) - 1)(a^(r/2) + 1) = kN.
Calculate p = GCD(a^(r/2) - 1, N) and q = GCD(a^(r/2) + 1, N).
p and q are likely to be non-trivial factors of N. The algorithm terminates, having (likely) found a factorization.

In [1]:
!pip install QuantumRingsLib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.5 MB/s eta 0:00:00


## Import the required modules and obtain the backend

In [2]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from matplotlib import pyplot as plt


provider = QuantumRingsProvider(token ='rings-200.ClLnaB7LhHsuphvAbGRIa1to7KDTMGzt', name='shiva_sj@ph.iitr.ac.in')
backend = provider.get_backend("scarlet_quantum_rings")
provider.active_account()

{'name': 'shiva_sj@ph.iitr.ac.in',
 'token': 'rings-200.ClLnaB7LhHsuphvAbGRIa1to7KDTMGzt',
 'max_qubits': '200'}

In [3]:


import random

import numpy as np
from math import pi, ceil, log, floor, sqrt


def swap_registers(circuit, reg, n):
	for i in range(n//2):
		circuit.swap(reg[i], reg[n-i-1])

# applies qft to first n qubits of reg
def qft(circuit, reg, n, swaps):
	for j in range(n):
		circuit.h(reg[n-j-1])
		for m in range(n-j-1):
			circuit.cu1(pi/float(2**(n-j-1-m)), reg[m], reg[n-j-1])

	if(swaps):
		swap_registers(circuit, reg, n)

# applies inverse qft to first n qubits of reg
def qft_dagger(circuit, reg, n, swaps):
	if(swaps):
		swap_registers(circuit, reg, n)

	for j in range(n):
		for m in range(j):
			circuit.cu1(-pi/float(2**(j-m)), reg[m], reg[j])
		circuit.h(reg[j])

##############################################################################################

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

"""Function that calculates the array of angles to be used in the addition in Fourier Space"""
def getAngles(a,N):
    s=bin(int(a))[2:].zfill(N)
    angles=np.zeros([N])
    for i in range(0, N):
        for j in range(i,N):
            if s[j]=='1':
                angles[N-i-1]+=pow(2, -(j-i))
        angles[N-i-1]*=np.pi
    return angles

"""Creation of a doubly controlled phase gate"""
def ccphase(circuit,angle,ctl1,ctl2,tgt):
    circuit.cu1(angle/2,ctl1,tgt)
    circuit.cx(ctl2,ctl1)
    circuit.cu1(-angle/2,ctl1,tgt)
    circuit.cx(ctl2,ctl1)
    circuit.cu1(angle/2,ctl2,tgt)

"""Creation of the circuit that performs addition by a in Fourier Space"""
"""Can also be used for subtraction by setting the parameter inv to a value different from 0"""
def phiADD(circuit,q,a,N,inv):
    angle=getAngles(a,N)
    for i in range(0,N):
        if inv==0:
            circuit.u1(angle[i],q[i])
        else:
            circuit.u1(-angle[i],q[i])

"""Single controlled version of the phiADD circuit"""
def cphiADD(circuit,q,ctl,a,n,inv):
    angle=getAngles(a,n)
    for i in range(0,n):
        if inv==0:
            circuit.cu1(angle[i],ctl,q[i])
        else:
            circuit.cu1(-angle[i],ctl,q[i])

"""Doubly controlled version of the phiADD circuit"""
def ccphiADD(circuit,q,ctl1,ctl2,a,n,inv):
    angle=getAngles(a,n)
    for i in range(0,n):
        if inv==0:
            ccphase(circuit,angle[i],ctl1,ctl2,q[i])
        else:
            ccphase(circuit,-angle[i],ctl1,ctl2,q[i])

"""Circuit that implements doubly controlled modular addition by a"""
def ccphiADDmodN(circuit, q, ctl1, ctl2, aux, a, N, n):
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 0)
    phiADD(circuit, q, N, n, 1)
    qft_dagger(circuit, q, n, 0)
    circuit.cx(q[n-1],aux)
    qft(circuit,q,n,0)
    cphiADD(circuit, q, aux, N, n, 0)

    ccphiADD(circuit, q, ctl1, ctl2, a, n, 1)
    qft_dagger(circuit, q, n, 0)
    circuit.x(q[n-1])
    circuit.cx(q[n-1], aux)
    circuit.x(q[n-1])
    qft(circuit,q,n,0)
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 0)

"""Circuit that implements the inverse of doubly controlled modular addition by a"""
def ccphiADDmodN_inv(circuit, q, ctl1, ctl2, aux, a, N, n):
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 1)
    qft_dagger(circuit, q, n, 0)
    circuit.x(q[n-1])
    circuit.cx(q[n-1],aux)
    circuit.x(q[n-1])
    qft(circuit, q, n, 0)
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 0)
    cphiADD(circuit, q, aux, N, n, 1)
    qft_dagger(circuit, q, n, 0)
    circuit.cx(q[n-1], aux)
    qft(circuit, q, n, 0)
    phiADD(circuit, q, N, n, 0)
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 1)

"""Circuit that implements single controlled modular multiplication by a"""
def cMULTmodN(circuit, ctl, q, aux, a, N, n):
    qft(circuit,aux,n+1,0)
    for i in range(0, n):
        ccphiADDmodN(circuit, aux, q[i], ctl, aux[n+1], (2**i)*a % N, N, n+1)
    qft_dagger(circuit, aux, n+1, 0)

    for i in range(0, n):
        circuit.cswap(ctl,q[i],aux[i])

    a_inv = modinv(a, N)
    qft(circuit, aux, n+1, 0)
    i = n-1
    while i >= 0:
        ccphiADDmodN_inv(circuit, aux, q[i], ctl, aux[n+1], pow(2,i)*a_inv % N, N, n+1)
        i -= 1
    qft_dagger(circuit, aux, n+1, 0)


# calculate period of f(x)=a^x mod N
def periodFinding(N, a):
	n = ceil(log(N, 2))
	if(n<4):
		n=4
	m = 2*n

	qregUp 		= QuantumRegister(m)
	qregDown 	= QuantumRegister(n)
	aux 		= QuantumRegister(n+2)
	creg 		= ClassicalRegister(m)

	qc = QuantumCircuit(qregUp, qregDown, aux, creg)

	qft(qc, qregUp, m, 1)
	qc.x(qregDown[0])

	for i in range(0, m):
		cMULTmodN(qc, qregUp[i], qregDown, aux, int(pow(a, pow(2, i))), N, n)

	qft_dagger(qc, qregUp, m, 1)
	qc.measure(qregUp, creg)


	shots = 1000
	# job = execute(qc, Aer.get_backend('qasm_simulator'), shots=shots)
	# counts = job.result().get_counts(qc)

	# Execute the circuit
	job = backend.run(qc, shots=shots) #Fixed indentation error
	job_monitor(job)
	result = job.result()
	counts = result.get_counts()


	### process data start
	i=0
	avg = 0
	while i < len(counts):
		val = list(counts.values())[i]
		output_desired = list(counts.keys())[i]
		x_value = int(output_desired, 2)
		print("value:{0} \t times:{1}".format(x_value, val))

		avg += (val*val)/shots

		i += 1

	i=0
	Variance = 0
	while i < len(counts):
		val = list(counts.values())[i]

		Variance += (((val-avg)**2)*val)/shots

		i += 1

	stdDev = sqrt(Variance)
	treshold = avg - stdDev

	if(stdDev<avg*0.3):
		treshold = 0

	print("---------------------------")
	print("avg     : {0}".format(avg))
	print("stdDev  : {0}".format(stdDev))
	print("treshold: {0}".format(treshold))
	print("---------------------------")
	### process data end


  #plot (generally not required)
	# plot = plt.bar(counts.keys(), counts.values())
	# plt.show()

	i=0
	MOverR = 2**m
	while i < len(counts):
		val = list(counts.values())[i]
		if(val<treshold):
			i+=1
			continue
		output_desired = list(counts.keys())[i]
		x_value = int(output_desired, 2)

		print("value:{0} \t times:{1}".format(x_value, val))

		if(x_value<MOverR and x_value!=0):
			MOverR = x_value

		i+=1;


	print("---------------------------")
  #clean up
  # del q, c, qc
	del qregUp
	del qregDown
	del aux
	del creg
	del qc
	del result
	del job
	return floor(((2**m)/MOverR)+0.5) #contFrac(MOverR, m)

def gcd(a, b):
	if(b==0):
		return a
	else:
		return gcd(b, a%b)

# select a random number a
def Step1(N):
	a = random.randint(2, N-1)
	print("Selected a: {0}".format(a))

	gcdAN = gcd(a,N)
	if(gcdAN!=1):
		p = gcdAN
		q = N/gcdAN
		return (p, q)
	else:
		return (a, 0)

# find the period and check if it satisfies the conditions
def Step2(N, a):
	print("executing the quantum circuit...")
	r = periodFinding(N, a)
	print("found r:{0}".format(r))
	if(r%2==1):
		return 0
	else:
		if((a**(r/2)+1)%N==0):
			print("a**(r/2)+1) mod N = 0")
			return 0
		else:
			return r


def run_shors_algorithm(N):
	r = 0
	a = 0
	p = 0
	q = 0
	while(True):
		print("-----------Step1--------------")
		(p,q) = Step1(N)
		if(q!=0):
			print("Lucky selection for a")
			return (p,q)
		else:
			a = p
			print("-----------Step2--------------")
			r = Step2(N,a)
			if(r!=0):
				p = gcd((a**(r/2)-1),N)
				q = gcd((a**(r/2)+1),N)
				break

			print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")

	return (p,q)

## executing the given example for first time

In [5]:
res = run_shors_algorithm(15)
print(res)

-----------Step1--------------
Selected a: 13
-----------Step2--------------
executing the quantum circuit...
Job Running
Job Running
Job Running
Job Done.
Ending Job Monitor
value:0 	 times:275
value:64 	 times:253
value:128 	 times:249
value:192 	 times:223
---------------------------
avg     : 251.36400000000003
stdDev  : 18.30594176763381
treshold: 0
---------------------------
value:0 	 times:275
value:64 	 times:253
value:128 	 times:249
value:192 	 times:223
---------------------------
found r:4
(3.0, 5.0)


## now let's execute oour algorithm on real tasks to be done.

In [6]:
semiprimes = {
    8: 143,
    10: 899,
    12: 3127,
    14: 11009,
    16: 47053,
    18: 167659,
    20: 744647,
    22: 3036893,
    24: 11426971,
    26: 58949987,
    28: 208241207,
    30: 857830637,
    32: 2776108693,
    34: 11455067797,
    36: 52734393667,
    38: 171913873883,
    40: 862463409547
}

#### as it is cumbersome lets remove print statements

In [10]:
# calculate period of f(x)=a^x mod N
def periodFinding(N, a):
	n = ceil(log(N, 2))
	if(n<4):
		n=4
	m = 2*n

	qregUp 		= QuantumRegister(m)
	qregDown 	= QuantumRegister(n)
	aux 		= QuantumRegister(n+2)
	creg 		= ClassicalRegister(m)

	qc = QuantumCircuit(qregUp, qregDown, aux, creg)

	qft(qc, qregUp, m, 1)
	qc.x(qregDown[0])

	for i in range(0, m):
		cMULTmodN(qc, qregUp[i], qregDown, aux, int(pow(a, pow(2, i))), N, n)

	qft_dagger(qc, qregUp, m, 1)
	qc.measure(qregUp, creg)


	shots = 1000
	# job = execute(qc, Aer.get_backend('qasm_simulator'), shots=shots)
	# counts = job.result().get_counts(qc)

	# Execute the circuit
	job = backend.run(qc, shots=shots) #Fixed indentation error
	# job_monitor(job)
	result = job.result()
	counts = result.get_counts()


	### process data start
	i=0
	avg = 0
	while i < len(counts):
		val = list(counts.values())[i]
		output_desired = list(counts.keys())[i]
		x_value = int(output_desired, 2)
		# print("value:{0} \t times:{1}".format(x_value, val))

		avg += (val*val)/shots

		i += 1

	i=0
	Variance = 0
	while i < len(counts):
		val = list(counts.values())[i]

		Variance += (((val-avg)**2)*val)/shots

		i += 1

	stdDev = sqrt(Variance)
	treshold = avg - stdDev

	if(stdDev<avg*0.3):
		treshold = 0

	# print("---------------------------")
	# print("avg     : {0}".format(avg))
	# print("stdDev  : {0}".format(stdDev))
	# print("treshold: {0}".format(treshold))
	# print("---------------------------")
	# ### process data end


  #plot (generally not required)
	# plot = plt.bar(counts.keys(), counts.values())
	# plt.show()

	i=0
	MOverR = 2**m
	while i < len(counts):
		val = list(counts.values())[i]
		if(val<treshold):
			i+=1
			continue
		output_desired = list(counts.keys())[i]
		x_value = int(output_desired, 2)

		# print("value:{0} \t times:{1}".format(x_value, val))

		if(x_value<MOverR and x_value!=0):
			MOverR = x_value

		i+=1;


	# print("---------------------------")
  #clean up
  # del q, c, qc
	del qregUp
	del qregDown
	del aux
	del creg
	del qc
	del result
	del job
	return floor(((2**m)/MOverR)+0.5) #contFrac(MOverR, m)



# select a random number a
def Step1(N):
	a = random.randint(2, N-1)
	# print("Selected a: {0}".format(a))

	gcdAN = gcd(a,N)
	if(gcdAN!=1):
		p = gcdAN
		q = N/gcdAN
		return (p, q)
	else:
		return (a, 0)

# find the period and check if it satisfies the conditions
def Step2(N, a):
	# print("executing the quantum circuit...")
	r = periodFinding(N, a)
	# print("found r:{0}".format(r))
	if(r%2==1):
		return 0
	else:
		if((a**(r/2)+1)%N==0):
			# print("a**(r/2)+1) mod N = 0")
			return 0
		else:
			return r


def run_shors_algorithm(N):
	r = 0
	a = 0
	p = 0
	q = 0
	while(True):
		# print("-----------Step1--------------")
		(p,q) = Step1(N)
		if(q!=0):
			# print("Lucky selection for a")
			return (p,q)
		else:
			a = p
			# print("-----------Step2--------------")
			r = Step2(N,a)
			if(r!=0):
				p = gcd((a**(r/2)-1),N)
				q = gcd((a**(r/2)+1),N)
				break

			# print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")

	return (p,q)

In [15]:
res = run_shors_algorithm(semiprimes[8])
print(res)

(11, 13.0)


In [17]:
res = run_shors_algorithm(semiprimes[10])
print(res)

(31, 29.0)


In [18]:
res = run_shors_algorithm(semiprimes[12])
print(res)

(59, 53.0)


###  after this it takes too much time to compute

# Refrences

https://arxiv.org/pdf/quant-ph/0205095

# END

